In [11]:
#importing the libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
#Creating the model:
recognizer = Sequential()

In [3]:
#Creating the first 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',
                             kernel_initializer='glorot_uniform', bias_initializer='zeros', input_shape=(256,256,3)))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [4]:
#Creating the second 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',
                             kernel_initializer='glorot_uniform', bias_initializer='zeros'))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [5]:
#Creating the third 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',
                             kernel_initializer='glorot_uniform', bias_initializer='zeros'))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [6]:
#Creating the forth 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',
                             kernel_initializer='glorot_uniform', bias_initializer='zeros'))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [7]:
#Transforming the pooling layer into a vector where each vector vlue = neuron
recognizer.add(Flatten())

In [8]:
#Creating the Artificial Neural Network
recognizer.add(Dense(input_dim = 256, units=128, activation='relu'))
recognizer.add(Dense(input_dim =128, units= 128, activation='relu'))
recognizer.add(Dense(input_dim =128, units= 128, activation='relu'))
recognizer.add(Dense(input_dim =128, units= 1, activation='sigmoid'))

In [9]:
#Compiling the Neural Network
recognizer.compile(optimizer = "adagrad" , loss="binary_crossentropy" , metrics=['accuracy'])

### Before continuing to training my model, applying k-folds and fitting on the test set I wanted make sure that my model has the abillity to overfit. The reason is to make sure that my data has enough features for the model to learn from and that my model extracts enough features to learn from.

In [17]:
path = os.path.abspath(os.getcwd())
#image = load_img('sets/val')
#input_arr = img_to_array(image)
#input_arr = np.array([input_arr])  # Convert single image to a batch.
#predictions = recognizer.fit(input_arr)
print(path)

C:\Users\User\Documents\GitHub\Pneumonia_Recognizer


In [ ]:
val_gen = ImageDataGenerator(rescale=1./255)
val_set = val_gen.flow_from_directory(
                                       target_size=(256, 256),
                                       batch_size=1,
                                       class_mode='binary')  

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
                                    
training_set = train_datagen.flow_from_directory('sets/train',
                                                  target_size=(256, 256),
                                                  batch_size=20,
                                                  class_mode='binary') 

Found 5216 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

testing_set = test_datagen.flow_from_directory('sets/validation',
                                                   target_size=(256, 256),
                                                   batch_size=2,
                                                   class_mode='binary')  